In [1]:
import numpy as np
import scipy.linalg
import functools
import cvxpy as cp
inv = scipy.linalg.inv 
randn = np.random.randn
from numpy.linalg import matrix_rank
from scipy.linalg import logm, expm
from scipy.linalg import sqrtm
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import math
from cvxpy.atoms.axis_atom import AxisAtom
from cvxpy.atoms.norm1 import norm1
from cvxpy.atoms.norm_inf import norm_inf
from cvxpy.constraints.constraint import Constraint
from cvxpy.utilities.power_tools import pow_high, pow_mid, pow_neg
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
# ! sudo apt-get install texlive-latex-recommended 
# ! sudo apt install texlive-latex-extra
# ! sudo apt install dvipng
# ! sudo apt install cm-super
from matplotlib import rc
from scipy.linalg import expm, sinm, cosm, logm
from scipy.stats import unitary_group
from scipy.optimize import fsolve

n1 = 6 ### number of data points for SNIP based {freq, unitary_matrix}
n2 = 6 ### number of data points for Geodesic Interpolation

In [2]:
def poly_add(m,p1,p2):
    l1 = p1.shape[0]
    l2 = p2.shape[0]
    if(l1 == l2):
        return p1 + p2
    elif(l1 > l2):
        p2_mod = np.zeros(p1.shape,dtype = np.complex128)
        p2_mod[l1-l2:l1] = p2
        return p1 + p2_mod
    else:
        p1_mod = np.zeros(p2.shape,dtype = np.complex128)
        p1_mod[l2-l1:l2] = p1
        return p1_mod + p2

### computing the coefficients of product of two polynomial (as a convolution of their coefficient vectors) 
def poly_convolve(m,p1,p2):
    if(m == 1):
        return np.convolve(p1,p2)
    else:
        result = []
        l1 = p1.shape[0]
        l2 = p2.shape[0]
        p1_mod = np.zeros((l1+2*(l2-1),m,m), dtype = np.complex128)# append zeroes

        #### zero padded p1
        for i in range(l1 + l2 - 1):
            if (i < l2 -1):
                p1_mod[i] = 0
            elif (i < l1 + l2 - 1):
                p1_mod[i] = p1[i - (l2 - 1)]
            else :
                p1_mod[i] = 0
                
        ### convlution of p1 & p2
        for i in range(l1 + l2 - 1):
            temp = np.zeros((m,m), dtype = np.complex128)
            for j in range(l2):
                temp = temp + p1_mod[i+j]@p2[l2-1-j]
            result.append(temp)
        return np.array(result)

In [3]:
### Evaluating the polynomial matrix at a particular frequency
def poly_eval(m,p,omega):
    l = p.shape[0]
    ans = np.zeros((m,m),dtype = np.complex128)
    s = omega
    for i in range(l):
        ans = ans + p[l-1-i]*(s**i)
    return ans

In [4]:
### For testing on modified data
#### Unitary Transformation
def Modify_U(U, Flag):
  row = U[0]
  diagonal = np.diag(U)
  d_array = [(np.abs(x))/x for x in row]
  l = len(d_array)
  if Flag == 0:### No modification
    return U
  elif Flag == 1: #### Real First row
    D = np.diag(d_array)
  elif Flag == 2: #### Real first row with alternate signs
    for i  in range(l):
      d_array[i] = ((-1)**i ) * d_array[i]
    D = np.diag(d_array)
  elif Flag == 3 : ### real Diagonal Elements
    d_array = [(np.abs(x))/x for x in diagonal]
    D = np.diag(d_array)
  elif Flag == 4 : ### real Diagonal Elements alternate signs
    d_array = [(np.abs(x))/x for x in diagonal]
    l  = len(d_array)
    for i  in range(l):
      d_array[i] = ((-1)**i ) * d_array[i]
    D = np.diag(d_array)
  return U@D

In [5]:
### Base step in induaction based solution construction
def base_interpolation_poly(m,omega,U,Gamma):
    I = np.eye(m)
    N = [(I+ (U @ inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) ) ,(-1*omega*I+ (U @ inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) )]
    D = [(I+ (inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) )     ,   (-1*omega*I+ (inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) ) ]
    return np.array(N),np.array(D);

In [6]:
### Matrix used to modify the data in inductive step of Induction based solution construction 
def Transformation_mat_poly(m,omega_1,U1,Gamma_1): 
    I = np.eye(m)
    H = [np.array([2*I - (omega_1+1)*inv(Gamma_1), -2 * omega_1 * I - (omega_1 + 1)*inv(Gamma_1)]),
         np.array([(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T ,(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T]),
         np.array([(omega_1 + 1)*-1*U1 @ inv(Gamma_1),  (omega_1 + 1)*-1*U1 @ inv(Gamma_1)]),
         np.array([2*I + ((omega_1 + 1)* U1 @ inv(Gamma_1) @ U1.conj().T)   , (-2 * omega_1 * I) + ((omega_1 + 1) * U1 @ inv(Gamma_1) @ U1.conj().T) ])]
    return H

In [7]:
### Final step of combining the solutions (of N-1 points & 1 point) in Induction based solution construction {NOTE: This is done iteratively} 
def final_interpolation_poly(m,omega_1,U1,Gamma_1,Nz_hat,Dz_hat):
    I = np.eye(m)
    D1 = [2*I + (omega_1+1)*inv(Gamma_1),    -2 * omega_1 * I + (omega_1 + 1)*inv(Gamma_1)]
    D2 = [-1*(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T ,  -1*(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T]
    N1 = [2*I - (omega_1 + 1) *U1 @ inv(Gamma_1) @ U1.conj().T   , -2 * omega_1 * I - (omega_1 + 1) * U1 @ inv(Gamma_1) @ U1.conj().T]
    N2 = [(omega_1 + 1)*U1 @ inv(Gamma_1),  (omega_1 + 1)*U1 @ inv(Gamma_1)]

    Dz = poly_add(m,poly_convolve(m,np.array(D1),Dz_hat),poly_convolve(m,np.array(D2),Nz_hat))
    Nz = poly_add(m,poly_convolve(m,np.array(N1),Nz_hat),poly_convolve(m,np.array(N2),Dz_hat))
    return Nz, Dz;

In [8]:
def Recursive_interpolation_poly(n,m,omega_array,U_array,Gamma_array):
    I = np.eye(m)
    if(n != omega_array.shape[0]) or (n != U_array.shape[0]) or (n != Gamma_array.shape[0]):
        print("Incorrect no. of points in recursion")
        return
        
    if(n == 1): # Base step
        Nz_hat, Dz_hat = base_interpolation_poly(m,omega_array[0],U_array[0],Gamma_array[0])
        return Nz_hat, Dz_hat
    
    else: ### modify the data 
        omega = omega_array[0]
        U = U_array[0]
        Gamma = Gamma_array[0]

        ### modification for next step
        omega_array_mod = omega_array[1:n]
        eval_H = Transformation_mat_poly(m,omega,U,Gamma)
        U_array_mod = np.zeros((n-1,m,m), dtype = np.complex128)

        for i in range(n-1):
            ### Converting to polynomial
            H_omega_0 = poly_eval(m,eval_H[0],omega_array[i+1])
            H_omega_1 = poly_eval(m,eval_H[1],omega_array[i+1])
            H_omega_2 = poly_eval(m,eval_H[2],omega_array[i+1])
            H_omega_3 = poly_eval(m,eval_H[3],omega_array[i+1])        

            #### Modifying the data 'B hat'
            V_hat_1 = H_omega_0 + H_omega_1 @ U_array[i+1]
            V_hat_2 = H_omega_2 + H_omega_3 @ U_array[i+1]

            ### Computing "A hat"
            U_array_mod[i] =  V_hat_2 @ inv(V_hat_1)    

        Gamma_array_mod = np.zeros((n-1,m,m), dtype = np.complex128)
        
        for i in range(n-1):
            Gamma_array_mod[i] = Gamma_array[i+1] - (((I - np.conj(U_array[i+1].T)@U)@inv(Gamma)@(I - np.conj(U.T)@U_array[i+1]))/((1- (omega_array[i+1]/omega)) * (1- (omega/omega_array[i+1])) ))
        
        ### recursive
        Nz_hat, Dz_hat = Recursive_interpolation_poly(n-1,m,omega_array_mod,U_array_mod,Gamma_array_mod)
        
        ### combining
        Nz, Dz = final_interpolation_poly(m,omega,U,Gamma,Nz_hat,Dz_hat)
        
        return Nz, Dz

In [9]:
def Gs_eval(m,Nz,Dz,omega):
    return poly_eval(m,Nz,omega)@inv(poly_eval(m,Dz,omega))

In [10]:
####### FLAG DISTANCE METRIC FOR ERROR COMPUTATION
def flag_dist(V1,V2):
  V1 = V1.T
  V2 = V2.T
  r = len(V1)
  sum = 0
  for i in range(r):
    a = np.matrix(V1[i])
    b = np.matrix(V2[i])
    diff = a.getH()*a - b.getH()*b
    sum = sum + (np.linalg.norm(diff))**2
  d = np.sqrt(sum)
  return d

In [11]:
### generating a random hermitian positive definite matrix
def hermitian_PD_matrix(m):
    while 1:
        Gamma_1 = (randn(m,m) + 1j * randn(m,m))
        Gamma_1 = Gamma_1 + np.conj(Gamma_1.T) #### Hermitian
        if np.all(np.linalg.eigvals(Gamma_1) > 0):
            break
    return Gamma_1

In [12]:
###--> minimize the sum of diagonal elements of pick matrix, under constraint that, pick matrix is positive definite 
def Optimize_Gammas(n,m,omega_array,U_array):
    Gamma_array = np.zeros((n,m,m), dtype = np.complex128)
    for i in range(n):
        Gamma_array[i] = hermitian_PD_matrix(m)
        Gamma_array[i] = Gamma_array[i] - np.diag(np.diag(Gamma_array[i])) ### making diagonal elements of gamma as zeros
    
    # Defining Pick matrix
    P_n = np.zeros((n*m,n*m), dtype = np.complex128)
    I = np.eye(m)
    for i in range(n):
        for j in range(n):
            if(i == j):
                P_n[i*m:(i+1)*m,j*m:(j+1)*m] = Gamma_array[i]
            else:
                P_n[i*m:(i+1)*m,j*m:(j+1)*m] = (I - np.conj(U_array[i].T)@U_array[j])/(1-(omega_array[i] * (1/omega_array[j])))
    ###### convex optimization problem
    Pick_Diag = cp.Variable((n*m))

    #####  Constraint : Positive semi definite Pick Matrix
    constraints = [P_n + cp.diag(Pick_Diag) >> 0] 
    #### minimizing the sum of diagoanal elements of gamma(group delays)
    prob = cp.Problem(cp.Minimize(sum(Pick_Diag)),constraints)
    prob.solve()

    ## checking positive definiteness of pick matrix(since strict PD cannot be constrained in cvxpy)
    P_n = P_n + np.diag(Pick_Diag.value)
    if not np.all(np.linalg.eigvals(P_n) > 0):
        Gamma_array = Optimize_Gammas(n,m,omega_array,U_array)
    else:
      ### constructing optimal Gamma_matrices    
      for i in range(n):
          Gamma_array[i] = Gamma_array[i] + np.diag(np.diag(P_n[i*m:(i+1)*m,i*m:(i+1)*m]))
    return Gamma_array

In [13]:
def SNIP_boundary(m,n,a,power_delay_profile, delay_profile,channel_cont_omega,Flag,plot_points,U_array_ip,omega_array_ip,scale,optimize):
  Gamma_array_ip = np.zeros((n,m,m), dtype = complex)
  delta_w = 10**(-12)
  # print(n)
  for i in range(n):
    if optimize == 1:
      Gamma_array_ip = Optimize_Gammas(n,m,omega_array_ip,U_array_ip)
    elif optimize == 0:
      U,S,Vh = scipy.linalg.svd(channel_cont_omega(np.imag(np.log(omega_array_ip[i])) + delta_w))
      U_w_plus_delta = U
      Phi_w_plus_delta = -1j*logm(U_w_plus_delta)
      Phi_w_plus_delta_Her = (Phi_w_plus_delta + np.conj(Phi_w_plus_delta.T))/2

      if not np.allclose(Phi_w_plus_delta,(Phi_w_plus_delta + np.conj(Phi_w_plus_delta.T))/2):
          print('Phi_w_plus_delta is not Hermitian for i =',i)    

      U_w = U_array_ip[i]

      Phi_w = -1j*logm(U_w) ##### Phase ???
      Phi_w_Her = (Phi_w + np.conj(Phi_w.T))/2

      if not np.allclose(Phi_w,Phi_w_Her):
          print('Phi_w is not Hermitian for i =',i)

      ### since delta_w is already scaled, we have to un-scale it before deividing below
      delta_w = delta_w*scale
      differentiate_phi = (Phi_w_plus_delta_Her - Phi_w_Her)/delta_w  #### differential of phase

      Gamma = 2*sqrtm(np.conj(differentiate_phi.T)@differentiate_phi) ## square root of matrix product
      gamma_her = np.conj(Gamma.T) - Gamma

      if not np.allclose(gamma_her,0):
        print('Gamma is not Hermitian')
      
      if not np.all(np.linalg.eigvals(Gamma) > 0):
          print('Gamma is Not PSD')
      Gamma_array_ip[i] = Gamma
    Group_delays = Gamma_array_ip
  ### We want to compute the interpolation at original omega values(i.e unscaled) , so we have to unscale the omega values before passing them through Recursive Interpolation
  for q in range(len(omega_array_ip)):
    omega_array_ip[q] = np.exp((scale)*np.log(omega_array_ip[q]))

  Nz, Dz = Recursive_interpolation_poly(n,m,omega_array_ip,U_array_ip,Gamma_array_ip)
  return Nz,Dz

In [14]:
# 4x4 simulation plot, ITU_Vehicular_A
no_of_plots = 3
channel_samples = 100
plot_points = 60
plot_err_3d_array = np.zeros((channel_samples,no_of_plots,plot_points))
plot_err = np.zeros((no_of_plots,plot_points))
flag_err = np.zeros((no_of_plots,plot_points))
flag_err_3d_array = np.zeros((channel_samples,no_of_plots,plot_points))
# scale = 1e3
scale = 1
avg_plot_err_3d_array = np.zeros((channel_samples,no_of_plots,plot_points))
avg_flag_err_3d_array = np.zeros((channel_samples,no_of_plots,plot_points))

for channel_index in range(channel_samples):
    print('---------------channel Index is ---', channel_index)
    for plot_index in range(no_of_plots):
        m = 4
        I = np.eye(m)
        H_array = np.zeros((5,m,m),dtype = complex)
        
        #### Rayleigh fading channel
        for i in range(5):
            H_array[i] = (randn(m,m) + 1j * randn(m,m))*(1/np.sqrt(2))
        ### Sampling frequency  = 10^7
        sampling_freq = 1e7
        a = 4.99*1e6/sampling_freq
        # a = (a/sampling_freq)*2*np.pi
        # a = -1*np.pi*0.99*1e-3

        power_delay_profile = np.array([0, -1 ,-9, -10, -15, -20]) # ITU_Vehicular_A channel #### attenuation over differnt paths
        delay_profile = 1e-2*np.array([0 ,300 ,700 ,1100 ,1700, 2500])  #### time delay in multiple paths
        channel_cont_omega = lambda omega : (H_array[0])*(10**(power_delay_profile[0]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[0])) +                                            (H_array[1])*(10**(power_delay_profile[1]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[1])) +                                            (H_array[2])*(10**(power_delay_profile[2]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[2])) +                                            (H_array[3])*(10**(power_delay_profile[3]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[3])) +                                            (H_array[4])*(10**(power_delay_profile[4]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[4]))

        omega_plot = np.zeros(plot_points , dtype = complex)

        U_plot = np.zeros((plot_points,m,m), dtype = complex)
        U_plot_real = np.zeros((plot_points,m,m), dtype = complex)

        for i in range(plot_points):
            omega_plot[i] = -a + 2*a*i/(plot_points-1)
            omega_plot[i] = np.exp(1j*omega_plot[i])
        
        normalization_const = 1

        omega_array_ip = np.zeros(n1,dtype = complex) # sampling rate = 10 MHz
        dum = int(plot_points/(n1-1))

        U_array_ip = np.zeros((n1,m,m), dtype = complex)
        
        if(plot_index == 0): # Our approach
            Flag = 0
            for i in range(plot_points):
              omega_plot[i] = -a + 2*a*i/(plot_points-1)
              omega_plot[i] = np.exp(1j*omega_plot[i])
              U,S,Vh = scipy.linalg.svd(channel_cont_omega(np.imag(np.log(omega_plot[i]))))
              U_plot[i] = U

              if (i%dum == 0):
                w = int(i/dum)
                U_array_ip[w] = Modify_U(U,Flag)
                omega_array_ip[w] = omega_plot[i]
              if i == plot_points-1:
                U_array_ip[n1-1] =  Modify_U(U,Flag)
                omega_array_ip[n1-1] = omega_plot[i]
            
            ###### Boundary Interpolation step
            Nz,Dz = SNIP_boundary(m,n1,a,power_delay_profile, delay_profile,channel_cont_omega,Flag,plot_points,U_array_ip,omega_array_ip,scale,0)

            for i in range(plot_points):
#               print('Plot Point : ', i , '      Printing the G*G matrix')
              dumm = Gs_eval(m,Nz,Dz,omega_plot[i])
              dum2 = (np.conj(dumm).T)@dumm
              targ = U_plot[i]
              dum3 = (np.conj(targ).T)@targ
#               print('TARGET :::')
#               for q in range(m):
#                 print([dum3[q][k] for k in range(m)])
#               print('Interpolated :::')
#               for q in range(m):
#                 print([dum2[q][k] for k in range(m)])
#               print('---------------------------------------------------------------------------------------------')
              dum1 =(np.conj(dumm).T)@dumm -I
              g = np.allclose( dum1 , 0)

              ### we have to unscale omega_plot befor plugging them in Gs_eval , because omega_plot is computed from scaled version of 'a'
              omega_plot[i] = np.exp((scale)*np.log(omega_plot[i]))
              plot_err[plot_index,i] = normalization_const*np.linalg.norm(U_plot[i] - Gs_eval(m,Nz,Dz,omega_plot[i]))
              flag_err[plot_index,i] = flag_dist(U_plot[i],Gs_eval(m,Nz,Dz,omega_plot[i]) )


        if(plot_index == 1): # Our approach Optimized
            Flag = 0
            for i in range(plot_points):
              omega_plot[i] = -a + 2*a*i/(plot_points-1)
              omega_plot[i] = np.exp(1j*omega_plot[i])
              U,S,Vh = scipy.linalg.svd(channel_cont_omega(np.imag(np.log(omega_plot[i]))))
              U_plot[i] = U

              if (i%dum == 0):
                w = int(i/dum)
                U_array_ip[w] = Modify_U(U,Flag)
                omega_array_ip[w] = omega_plot[i]
              if i == plot_points-1:
                U_array_ip[n1-1] =  Modify_U(U,Flag)
                omega_array_ip[n1-1] = omega_plot[i]
            
            ###### Boundary Interpolation step
            Nz,Dz = SNIP_boundary(m,n1,a,power_delay_profile, delay_profile,channel_cont_omega,Flag,plot_points,U_array_ip,omega_array_ip,scale,1)

            for i in range(plot_points):
#               print('Plot Point : ', i , '      Printing the G*G matrix')
              dumm = Gs_eval(m,Nz,Dz,omega_plot[i])
              dum2 = (np.conj(dumm).T)@dumm
              targ = U_plot[i]
              dum3 = (np.conj(targ).T)@targ
#               print('TARGET :::')
#               for q in range(m):
#                 print([dum3[q][k] for k in range(m)])
#               print('Interpolated :::')
#               for q in range(m):
#                 print([dum2[q][k] for k in range(m)])
#               print('---------------------------------------------------------------------------------------------')
              dum1 =(np.conj(dumm).T)@dumm -I
              g = np.allclose( dum1 , 0)

              ### we have to unscale omega_plot befor plugging them in Gs_eval , because omega_plot is computed from scaled version of 'a'
              omega_plot[i] = np.exp((scale)*np.log(omega_plot[i]))
              plot_err[plot_index,i] = normalization_const*np.linalg.norm(U_plot[i] - Gs_eval(m,Nz,Dz,omega_plot[i]))
              flag_err[plot_index,i] = flag_dist(U_plot[i],Gs_eval(m,Nz,Dz,omega_plot[i]) )

        ### SVD of channel matrices
        omega_array_ip = np.zeros(n2, dtype = complex) # sampling rate = 10 MHz
        for i in range(n2):
          omega_array_ip[i] = -a + 2*a*i/(n2-1)
          omega_array_ip[i] =omega_array_ip[i]
        U_array_ip = np.zeros((n2,m,m), dtype = complex)
        
        for i in range(n2):
          U,S,Vh = scipy.linalg.svd(channel_cont_omega(omega_array_ip[i]))
          U_array_ip[i] = U

        if(plot_index == no_of_plots - 1):# Geodesic
            U_interpol_linear = np.zeros((plot_points,m,m), dtype = complex)
            for k in range(plot_points):
                #### Linear combination over the logarithmic space
                omega_plot[k] = -a + 2*a*k/(plot_points-1)
                U,S,Vh = scipy.linalg.svd(channel_cont_omega(omega_plot[k]))
                U_plot[k] = U

            dum = np.int(plot_points/(n2-1))

            for g in range(n2-1):
              for p in range(dum):
                V1 = U_plot[g*dum]
                V2 = U_plot[min(g*dum+dum ,plot_points-1)]
                #----> paper
                # theta =np.diag(np.diag(np.conj(V2.T)@V1))
                # for w in range(len(theta)):
                #   theta[w][w] = theta[w][w]/abs(theta[w][w])
                # M = inv(V1)@V2@theta
                # t =  (omega_plot[g*dum+p]-omega_plot[g*dum])/((omega_plot[min(g*dum+dum ,plot_points-1)]-omega_plot[g*dum]))
                # U_interpol_linear[g*dum+p] = V1@expm(t*logm(M))
                
                ###-----> Prof.kumar
                B = logm(np.conj(V1.T)@V2)
                t =  (omega_plot[g*dum+p]-omega_plot[g*dum])/((omega_plot[min(g*dum+dum ,plot_points-1)]-omega_plot[g*dum]))
                U_interpol_linear[g*dum+p] = V1@expm(t*B)
          
                ###---> OLD
                U_interpol_linear[-1] = U_plot[-1]
                # U_interpol_linear[g*dum+p] = expm(((omega_plot[min(g*dum+dum ,plot_points-1)]-omega_plot[g*dum+p])/(omega_plot[min(g*dum+dum ,plot_points-1)]-omega_plot[g*dum])*logm(U_plot[g*dum])) + 
                #                                   ((omega_plot[g*dum+p]-omega_plot[g*dum])/(omega_plot[min(g*dum+dum ,plot_points-1)]-omega_plot[g*dum])*logm(U_plot[min(g*dum+dum ,plot_points-1)])))
                          
                plot_err[plot_index,g*dum+p] = normalization_const*np.linalg.norm(U_plot[g*dum+p] - U_interpol_linear[g*dum+p])
                
                flag_err[plot_index,g*dum+p] = flag_dist(U_plot[g*dum+p] , U_interpol_linear[g*dum+p] )
        

    plot_err_3d_array[channel_index] = plot_err
    flag_err_3d_array[channel_index] = flag_err
avg_plot_err_3d_array = avg_plot_err_3d_array + plot_err_3d_array
avg_flag_err_3d_array = avg_flag_err_3d_array + flag_err_3d_array

---------------channel Index is --- 0


KeyboardInterrupt: 

In [ ]:
print('Unitary matrices in Data',U_array_ip)

In [ ]:
print('Frequencies in Data',omega_array_ip)

In [ ]:
avg_plot_err = np.average(avg_plot_err_3d_array,axis = 0)
# rc('text', usetex=True)
freq = omega_plot*2*np.pi*scale

plt.figure(figsize=(20,10))

plt.plot(freq, avg_plot_err[0],'^-',linewidth = 3,markersize= 10,label = 'SNIP Approach')
plt.plot(freq, avg_plot_err[2],'^-',linewidth = 3,markersize= 10,label = 'Geodesic Interpolation')
plt.plot(freq, avg_plot_err[1],'*-',linewidth = 3,markersize= 10,label = 'SNIP Approach Optimized')

plt.xlabel('xlabel', fontsize=40)
plt.xlabel(r'$\omega$ (rad/s)')
plt.ylabel('ylabel', fontsize=40)
plt.ylabel(r'$||\hat{\bf{V}}(e^{j\omega}) - {\bf{V}}(e^{j\omega})||_{F}$')

v = plt.axis()
plt.axis([-np.pi, np.pi, 0.0, v[3]])
xticks = [-np.pi, -3*np.pi / 4, -np.pi / 2, -np.pi / 4]
xticks = xticks + [0] + [-i for i in xticks[::-1]]
xtick_labels = [r'$-\pi$', r'$-\frac{3\pi}{4}$', r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$']
xtick_labels = xtick_labels + [r'$0$'] + [i.replace('-', '') for i in xtick_labels[::-1]]
plt.xticks(xticks, xtick_labels)
plt.xticks(fontsize=40)

# m = max(max(avg_plot_err_0),max(avg_plot_err_1))
plt.yticks(fontsize=32)
# major_ticks = np.arange(0, m, 0.03)
# plt.yticks(major_ticks,fontsize=32)

plt.grid(True, which = 'both')

plt.legend()
plt.legend(fontsize=24)
plt.savefig("frobenius_corrected.pdf",dpi = 5000)
plt.savefig("frobenius_corrected.jpg",dpi = 500)
plt.show()

In [ ]:
np.save('frequency', freq)
np.save('frobenius_SNIP',avg_plot_err[0])
np.save('frobenius_Geodesic', avg_plot_err[2])

In [ ]:
avg_flag_err =  np.average(avg_flag_err_3d_array,axis = 0)
# rc('text', usetex=True)
freq =omega_plot*2*np.pi*scale
# freq =omega_plot
plt.figure(figsize=(20,10))
plt.grid(True, which = 'both')

plt.plot(freq, avg_flag_err[0],'^-',linewidth = 3,markersize= 10,label = 'SNIP Approach')
plt.plot(freq, avg_flag_err[2],'^-',linewidth = 3,markersize= 10,label = 'Geodesic Interpolation')
plt.plot(freq, avg_flag_err[1],'*-',linewidth = 3,markersize= 10,label = 'SNIP Approach Optimized')

plt.xlabel('xlabel', fontsize=40)
plt.xlabel(r'$\omega$ (rad/s)')
plt.ylabel('ylabel', fontsize=40)
plt.ylabel(r'Flag Distance$[\hat{\bf{V}}(e^{j\omega}) , {\bf{V}}(e^{j\omega})]$')

v = plt.axis()
plt.axis([-np.pi, np.pi, 0.0, v[3]])
xticks = [-np.pi, -3*np.pi / 4, -np.pi / 2, -np.pi / 4]
xticks = xticks + [0] + [-i for i in xticks[::-1]]
xtick_labels = [r'$-\pi$', r'$-\frac{3\pi}{4}$', r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$']
xtick_labels = xtick_labels + [r'$0$'] + [i.replace('-', '') for i in xtick_labels[::-1]]
plt.xticks(xticks, xtick_labels)
plt.xticks(fontsize=40)

# m = max(max(avg_flag_err_0),max(avg_flag_err_1))
plt.yticks(fontsize=32)
# major_ticks = np.arange(0, m, 0.03)
# plt.yticks(major_ticks,fontsize=32)

plt.grid(True, which = 'both')

plt.legend()
plt.legend(fontsize=24)
plt.savefig("flag_corrected.pdf",dpi = 5000)
plt.savefig("flag_corrected.jpg",dpi = 500)
# plt.show()

In [ ]:
np.save('frequency', freq)
np.save('flag_SNIP',avg_flag_err[0])
np.save('flag_Geodesic', avg_flag_err[2])